In [1]:
using LinearAlgebra, Distributions, CSV, DataFrames, ProgressMeter
include("../../slv.jl")
using .slv
include("../../equation.jl")
using .equation
include("../../lsmc_bsde.jl")
using .lsmc_bsde

In [2]:
T = 1.0; r=0.01; S0 = 100.0; v0 = 0.4; α = 0.65; β = 0.7; ρ = 0.65; K = 100.0; η=0.9; θ₀=0.02;
slvm = HestonSABRSLV(S0::Float64, v0::Float64, ρ::Float64, r::Float64, β::Float64, η::Float64, α::Float64, θ₀::Float64);
Π = [1.0 ρ; ρ 1.0]; cholΠ=cholesky(Π).L

drift(x) = [slvm.ω(abs(x[1]),abs(x[2])), slvm.μ(abs(x[2]))]
diffusion(x) = Diagonal(
        [slvm.m(abs(x[2]))*slvm.Γ(abs(x[1])), slvm.σ(abs(x[2]))]
        )*cholΠ
R = 0.07;
driver(t, x, y, z) = (
    -r*max(y-z'*inv(diffusion(x))*x, 0.0)
    -R*min(y-z'*inv(diffusion(x))*x, 0.0)
    -z'*inv(diffusion(x))*drift(x)
)
terminal(x) = max(K-x[1],0)
bsde = BSDE(T, [S0, v0], drift, diffusion, driver, terminal);

In [3]:
#### LSMC
#### Laguerre polynomials
function Laguerre1D(x,k)
    if k == 0
        return 1.0
    elseif k == 1
        return (1.0-x)
    elseif k == 2
        return (x^2-4*x+2)/2
    elseif k == 3
        return (-x^3+9*x^2-18*x+6)/6
    elseif k == 4
        return (x^4-16*x^3+72*x^2-96*x+24)/24
    elseif k == 5
        return (-x^5+25*x^4-200*x^3+600*x^2-600*x+120)/120
    elseif k == 6
        return (x^6-36*x^5+450*x^4-2400*x^3+5400*x^2-4320*x+720)/720
    elseif k == 7
        return (-x^7+49*x^6-882*x^5+7350*x^4-29400*x^3+52920*x^2-35280*x+5040)/5040
    elseif k == 8
        return (x^8-64*x^7+1568*x^6-18816*x^5+117600*x^4-376320*x^3+564480*x^2-322560*x+40320)/40320
    elseif k == 9
        return (-x^9+81*x^8-2592*x^7+42336*x^6-381024*x^5+1905120*x^4-5080320*x^3+6531840*x^2-3265920*x+362880)/362880
    else
        return 0.0
    end
end

function bases2D(x, poly)
    vec = zeros((poly+1)^2)
    for k = 0:poly
        for l = 0:poly
            vec[(poly-1)*k+l+1] = Laguerre1D(x[1], k)*Laguerre1D(x[2], l)
        end
    end
    return vec
end

M=2^20

1048576

In [5]:
s0 = 100.0
bsde = BSDE(T, [s0, v0], drift, diffusion, driver, terminal);
samples = 50;
@showprogress for N in [6,7,8,9]
    poly = N
    print(string("N=", N,":\n")); flush(stdout)
    ba = (x->bases2D(x, poly))
    #### Generate 50 independent samples
    Y0 = zeros(samples);
    exc_start = time()
    @showprogress for k in 1:samples
        Y0[k] = LSMC(bsde, ba, M, N);
    end
    exc_stop = time()
    elapsed_time = exc_stop-exc_start
    print(string(":LSMCY_0(",s0,", ",v0,")=",mean(Y0),"/std=",std(Y0),". (",elapsed_time," seconds)\n")); flush(stdout)
end

N=6:


Progress: 100%|█████████████████████████████████████████| Time: 0:54:10


:LSMCY_0(100.0, 0.4)=5.595603184254965/std=0.015305921265513221. (3250.837821960449 seconds)
N=7:


Progress: 100%|█████████████████████████████████████████| Time: 1:18:57


:LSMCY_0(100.0, 0.4)=5.613204322339325/std=0.01532673147428293. (4737.513890981674 seconds)


...

Progress:  75%|██████████████████████████████▊          |  ETA: 1:22:06

N=9:


Progress: 100%|█████████████████████████████████████████| Time: 2:44:23


:LSMCY_0(100.0, 0.4)=5.62485025869727/std=0.0191995129964935. (9863.743056058884 seconds)


Progress: 100%|█████████████████████████████████████████| Time: 6:50:40
